In [1]:
import pandas as pd

In [2]:
import pandas as pd
import os
import joblib

In [3]:
print(" Loading merged dataset from Parquet...")
mimic_df = pd.read_parquet("merged_mimic_data.parquet")
print(f" Loaded dataset with {mimic_df.shape[0]:,} rows and {mimic_df.shape[1]} columns.")

 Loading merged dataset from Parquet...
 Loaded dataset with 116,426 rows and 42 columns.


In [4]:
selected_features = ['age', 'admissionweight', 'respiratoryrate', 'ph', 'bun',
       'glucose', 'motor',
       'mean_BUN', 'mean_Hgb', 'mean_WBC x 1000', 'mean_chloride',
       'mean_creatinine', 'mean_glucose', 'mean_lactate', 'mean_pH',
       'mean_paCO2', 'mean_paO2', 'mean_platelets x 1000', 'mean_potassium',
       'mean_sodium', 'mean_total bilirubin', 'heartrate', 'respiration',
        'sao2', 'systemicdiastolic', 'systemicmean',
       'systemicsystolic', 'outputtotal_mean']

X = mimic_df[selected_features]
y = mimic_df["actualhospitalmortality"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [5]:
len(selected_features)

28

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report
import numpy as np

def evaluate_model_pretty(name, y_test, y_pred, y_prob, y, n_classes, results):
    if n_classes == 2:
        if y_prob.ndim == 2 and y_prob.shape[1] == 2:
            roc_auc = roc_auc_score(y_test, y_prob[:, 1])
        else:
            roc_auc = roc_auc_score(y_test, y_prob.flatten())
    else:
        roc_auc = roc_auc_score(
            label_binarize(y_test, classes=np.unique(y)),
            y_prob,
            average='macro',
            multi_class='ovo'
        )

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')

    print(f"\n{'='*80}")
    print(f"Evaluation of {name} on Test Set:")
    print(f"  Accuracy : {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall   : {rec:.4f}")
    print(f"  AUC-ROC  : {roc_auc:.4f}\n")
    print("Detailed Report:\n")
    print(classification_report(y_test, y_pred, digits=2))
    print(f"{'='*80}")

    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "ROC-AUC": roc_auc
    })


results = []
n_classes = len(np.unique(y_train))
y = np.concatenate([y_train, y_test])  

In [7]:
logr_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=2000))
])
logr_model.fit(X_train, y_train)

y_prob_logr = logr_model.predict_proba(X_test)[:, 1]

y_pred_logr = (y_prob_logr > 0.3).astype(int)

# Evaluate
evaluate_model_pretty("Logistic Regression ", y_test, y_pred_logr, y_prob_logr, y, n_classes, results)


Evaluation of Logistic Regression  on Test Set:
  Accuracy : 0.9268
  Precision: 0.7692
  Recall   : 0.6822
  AUC-ROC  : 0.8552

Detailed Report:

              precision    recall  f1-score   support

           0       0.95      0.98      0.96     21346
           1       0.59      0.39      0.47      1940

    accuracy                           0.93     23286
   macro avg       0.77      0.68      0.72     23286
weighted avg       0.92      0.93      0.92     23286



In [8]:
nb_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', GaussianNB())
])
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
y_prob_nb = nb_model.predict_proba(X_test)

y_prob_nb = nb_model.predict_proba(X_test)[:, 1]

y_pred_nb = (y_prob_nb > 0.3).astype(int)

evaluate_model_pretty("Gaussian Naive Bayes", y_test, y_pred_nb, y_prob_nb, y, n_classes, results)


Evaluation of Gaussian Naive Bayes on Test Set:
  Accuracy : 0.8408
  Precision: 0.6169
  Recall   : 0.7163
  AUC-ROC  : 0.8021

Detailed Report:

              precision    recall  f1-score   support

           0       0.96      0.87      0.91     21346
           1       0.28      0.57      0.37      1940

    accuracy                           0.84     23286
   macro avg       0.62      0.72      0.64     23286
weighted avg       0.90      0.84      0.86     23286



In [9]:
knn_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier(n_neighbors=5))
])
knn_model.fit(X_train, y_train)

y_prob_knn = knn_model.predict_proba(X_test)[:, 1]

y_pred_knn = (y_prob_knn > 0.3).astype(int)

evaluate_model_pretty("K-Nearest Neighbors ", y_test, y_pred_knn, y_prob_knn, y, n_classes, results)


Evaluation of K-Nearest Neighbors  on Test Set:
  Accuracy : 0.9066
  Precision: 0.6993
  Recall   : 0.7183
  AUC-ROC  : 0.8463

Detailed Report:

              precision    recall  f1-score   support

           0       0.95      0.94      0.95     21346
           1       0.45      0.49      0.47      1940

    accuracy                           0.91     23286
   macro avg       0.70      0.72      0.71     23286
weighted avg       0.91      0.91      0.91     23286



In [10]:
svc_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', CalibratedClassifierCV(LinearSVC(max_iter=1000), cv=3))
])
svc_model.fit(X_train, y_train)
y_pred_svc = svc_model.predict(X_test)
y_prob_svc = svc_model.predict_proba(X_test)

y_prob_svc = y_prob_svc[:, 1]
y_pred_svc = (y_prob_svc > 0.3).astype(int)

evaluate_model_pretty("LinearSVC + Calibration", y_test, y_pred_svc, y_prob_svc, y, n_classes, results)

/Users/parthshiroya/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/parthshiroya/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/parthshiroya/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/parthshiroya/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/parthshiroya/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of


Evaluation of LinearSVC + Calibration on Test Set:
  Accuracy : 0.9280
  Precision: 0.7839
  Recall   : 0.6596
  AUC-ROC  : 0.8544

Detailed Report:

              precision    recall  f1-score   support

           0       0.94      0.98      0.96     21346
           1       0.63      0.34      0.44      1940

    accuracy                           0.93     23286
   macro avg       0.78      0.66      0.70     23286
weighted avg       0.92      0.93      0.92     23286



/Users/parthshiroya/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

print(" Training Random Forest with Class Balancing...")

rf_model = RandomForestClassifier(
    n_estimators=100, max_depth=10, 
    random_state=42, n_jobs=-1
)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)  

y_prob_rf = y_prob_rf[:, 1]

threshold = 0.3
y_pred_rf = (y_prob_rf > threshold).astype(int)

evaluate_model_pretty("Random Forest (Balanced)", y_test, y_pred_rf, y_prob_rf, y, n_classes, results)

 Training Random Forest with Class Balancing...

Evaluation of Random Forest (Balanced) on Test Set:
  Accuracy : 0.9354
  Precision: 0.8214
  Recall   : 0.6920
  AUC-ROC  : 0.9045

Detailed Report:

              precision    recall  f1-score   support

           0       0.95      0.98      0.97     21346
           1       0.70      0.40      0.51      1940

    accuracy                           0.94     23286
   macro avg       0.82      0.69      0.74     23286
weighted avg       0.93      0.94      0.93     23286



In [12]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

# Train a simple XGBoost model (default settings)
xgb_model_simple = xgb.XGBClassifier(random_state=42)
xgb_model_simple.fit(X_train, y_train)

# Generate predictions
y_pred_xgb_simple = xgb_model_simple.predict(X_test)
y_prob_xgb_simple = xgb_model_simple.predict_proba(X_test)[:, 1]  # Positive class probability

y_pred_xgb_simple = (y_prob_xgb_simple > 0.3).astype(int)

# Use evaluate_model_pretty for consistent evaluation
evaluate_model_pretty("XGBoost (Default Params)", y_test, y_pred_xgb_simple, y_prob_xgb_simple, y, n_classes, results)



Evaluation of XGBoost (Default Params) on Test Set:
  Accuracy : 0.9301
  Precision: 0.7718
  Recall   : 0.7641
  AUC-ROC  : 0.9414

Detailed Report:

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21346
           1       0.58      0.56      0.57      1940

    accuracy                           0.93     23286
   macro avg       0.77      0.76      0.77     23286
weighted avg       0.93      0.93      0.93     23286



In [13]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, roc_auc_score

print(" Training XGBoost with Best Hyperparameters...")

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=1,
    min_child_weight=1,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)


y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]

threshold = 0.3
y_pred_xgb = (y_prob_xgb > threshold).astype(int)

evaluate_model_pretty("XGBoost (0.3 threshold)", y_test, y_pred_xgb, y_prob_xgb, y, n_classes, results)

 Training XGBoost with Best Hyperparameters...

Evaluation of XGBoost (0.3 threshold) on Test Set:
  Accuracy : 0.9316
  Precision: 0.7768
  Recall   : 0.7710
  AUC-ROC  : 0.9474

Detailed Report:

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21346
           1       0.59      0.58      0.58      1940

    accuracy                           0.93     23286
   macro avg       0.78      0.77      0.77     23286
weighted avg       0.93      0.93      0.93     23286



In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # 2D
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

pos_weight_value = (len(y_train_tensor) - y_train_tensor.sum()) / y_train_tensor.sum()
pos_weight = torch.tensor([pos_weight_value.item()])


train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=512, shuffle=True)


class ICU_NN(nn.Module):
    def __init__(self, input_size):
        super(ICU_NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)  


model = ICU_NN(X_train.shape[1])
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)


print(" Training Neural Network with Class Weighting...")
for epoch in range(10):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/10 - Loss: {loss.item():.4f}")


print("Training Completed!")
with torch.no_grad():
    logits = model(X_test_tensor)
    probs = torch.sigmoid(logits) 
    y_pred = (probs > 0.3).float()

    y_test_np = y_test_tensor.numpy().astype(int).flatten()
    y_pred_np = y_pred.numpy().astype(int).flatten()
    y_prob_np = probs.numpy()  

    evaluate_model_pretty("Neural Network (PyTorch)", y_test_np, y_pred_np, y_prob_np, y, n_classes, results)

 Training Neural Network with Class Weighting...
Epoch 1/10 - Loss: 0.7433
Epoch 2/10 - Loss: 0.9097
Epoch 3/10 - Loss: 0.7943
Epoch 4/10 - Loss: 0.9005
Epoch 5/10 - Loss: 0.8806
Epoch 6/10 - Loss: 0.8571
Epoch 7/10 - Loss: 0.7585
Epoch 8/10 - Loss: 0.7295
Epoch 9/10 - Loss: 0.7547
Epoch 10/10 - Loss: 0.7388
Training Completed!

Evaluation of Neural Network (PyTorch) on Test Set:
  Accuracy : 0.6649
  Precision: 0.5914
  Recall   : 0.7891
  AUC-ROC  : 0.9007

Detailed Report:

              precision    recall  f1-score   support

           0       0.99      0.64      0.78     21346
           1       0.19      0.94      0.32      1940

    accuracy                           0.66     23286
   macro avg       0.59      0.79      0.55     23286
weighted avg       0.92      0.66      0.74     23286



In [15]:
nn_prob = y_prob_np.flatten()

ensemble_prob = (nn_prob + y_prob_xgb) / 2.0

ensemble_pred = (ensemble_prob > 0.3).astype(int)

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

accuracy = accuracy_score(y_test, ensemble_pred)
precision = precision_score(y_test, ensemble_pred)
recall = recall_score(y_test, ensemble_pred)
roc_auc = roc_auc_score(y_test, ensemble_prob)

print("Evaluation of Combined Ensemble Model on Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")
print("\nDetailed Report:\n")
print(classification_report(y_test, ensemble_pred))

Evaluation of Combined Ensemble Model on Test Set:
Accuracy: 0.8110
Precision: 0.2891
Recall: 0.8696
AUC-ROC: 0.9251

Detailed Report:

              precision    recall  f1-score   support

           0       0.99      0.81      0.89     21346
           1       0.29      0.87      0.43      1940

    accuracy                           0.81     23286
   macro avg       0.64      0.84      0.66     23286
weighted avg       0.93      0.81      0.85     23286



In [16]:

results_df = pd.DataFrame(results)
print(results_df)

                      Model  Accuracy  Precision    Recall   ROC-AUC
0      Logistic Regression   0.926780   0.769212  0.682173  0.855219
1      Gaussian Naive Bayes  0.840806   0.616904  0.716350  0.802104
2      K-Nearest Neighbors   0.906596   0.699315  0.718260  0.846326
3   LinearSVC + Calibration  0.927982   0.783908  0.659632  0.854437
4  Random Forest (Balanced)  0.935412   0.821419  0.692036  0.904537
5  XGBoost (Default Params)  0.930087   0.771775  0.764110  0.941392
6   XGBoost (0.3 threshold)  0.931633   0.776765  0.771045  0.947391
7  Neural Network (PyTorch)  0.664949   0.591416  0.789132  0.900748


In [17]:
joblib.dump(rf_model, "random_forest_mimic.pkl")
print("Random Forest model saved successfully!")

joblib.dump(logr_model, "logistic_regression_mimic.pkl")
print(" Logistic Regression model saved successfully!")

joblib.dump(nb_model, "naive_bayes_mimic.pkl")
print("Gaussian Naive Bayes model saved successfully!")

joblib.dump(knn_model, "knn_mimic.pkl")
print(" K-Nearest Neighbors model saved successfully!")

joblib.dump(svc_model, "linear_svc_calibrated_mimic.pkl")
print("LinearSVC + Calibration model saved successfully!")

# Save the model
joblib.dump(xgb_model_simple, "xgb_simple_model.pkl")
print(" XGBoost model saved as 'xgb_simple_model.pkl'")

joblib.dump(xgb_model, "xgb_mimic.pkl")
print(" XGBoost model saved successfully!")

torch.save(model.state_dict(), "mimic_mortality.pth")
print(" Neural Network model saved successfully!")

Random Forest model saved successfully!
 Logistic Regression model saved successfully!
Gaussian Naive Bayes model saved successfully!
 K-Nearest Neighbors model saved successfully!
LinearSVC + Calibration model saved successfully!
 XGBoost model saved as 'xgb_simple_model.pkl'
 XGBoost model saved successfully!
 Neural Network model saved successfully!


In [18]:
X.columns

Index(['age', 'admissionweight', 'respiratoryrate', 'ph', 'bun', 'glucose',
       'motor', 'mean_BUN', 'mean_Hgb', 'mean_WBC x 1000', 'mean_chloride',
       'mean_creatinine', 'mean_glucose', 'mean_lactate', 'mean_pH',
       'mean_paCO2', 'mean_paO2', 'mean_platelets x 1000', 'mean_potassium',
       'mean_sodium', 'mean_total bilirubin', 'heartrate', 'respiration',
       'sao2', 'systemicdiastolic', 'systemicmean', 'systemicsystolic',
       'outputtotal_mean'],
      dtype='object')

In [19]:
len(X.columns)

28